<a href="https://colab.research.google.com/github/Madhuram2901/AcWOC-MovieMusic/blob/main/MovMusic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install streamlit
!pip install pyngrok


In [26]:
%%writefile movie_music_recommender.py
import streamlit as st
import pandas as pd
import json
from difflib import get_close_matches
from typing import List, Dict, Union

class GenreRecommendationSystem:
    def __init__(self, movies_path: str, music_path: str):
        self.movies_df = self._load_movie_data(movies_path)
        self.music_genres_df = self._load_music_data(music_path)
        self.genre_mapping = self._create_genre_mapping()

    def _load_movie_data(self, path: str) -> pd.DataFrame:
        try:
            movies_df = pd.read_csv(path, usecols=['id', 'genres', 'title'])
            movies_df['genres'] = movies_df['genres'].apply(
                lambda x: [genre['name'] for genre in json.loads(x)]
            )
            return movies_df
        except Exception as e:
            raise ValueError(f"Error loading movie data: {e}")

    def _load_music_data(self, path: str) -> pd.DataFrame:
        try:
            music_df = pd.read_csv(path)
            required_columns = ['genres', 'track_names']
            missing_columns = [col for col in required_columns if col not in music_df.columns]
            if missing_columns:
                raise ValueError(f"Missing columns in music data: {missing_columns}")
            return music_df
        except Exception as e:
            raise ValueError(f"Error loading music data: {e}")

    def _create_genre_mapping(self) -> Dict[str, List[str]]:
        return {
            'Action': ['electronic', 'rock', 'epic', 'bass music', 'drum and bass'],
            'Adventure': ['world', 'cinematic', 'orchestral', 'folk'],
            'Fantasy': ['21st century classical', 'orchestral', 'cinematic', 'world'],
            'Science Fiction': ['electronic', 'ambient', 'abstract', 'abstract beats'],
            'Crime': ['dark ambient', 'abstract hip hop', 'trip hop'],
            'Drama': ['classical', 'acoustic', 'piano', 'ambient'],
            'Comedy': ['a cappella', 'pop', 'quirky', 'fun'],
            'Romance': ['acoustic', 'piano', 'vocal'],
            'Horror': ['dark ambient', 'abstract', 'atmospheric'],
            'Thriller': ['dark ambient', 'electronic', 'atmospheric'],
            'Animation': ['8-bit', 'fun', 'quirky', 'orchestral'],
            'Family': ['acoustic', 'pop', 'folk', 'fun']
        }

    def find_movie(self, movie_title: str) -> Union[None, pd.Series]:
        matches = self.movies_df[self.movies_df['title'].str.contains(movie_title, case=False, na=False)]
        return matches.iloc[0] if not matches.empty else None

    def get_related_music_genres(self, movie_genres: List[str]) -> List[str]:
        related_music_genres = set()
        for genre in movie_genres:
            if genre in self.genre_mapping:
                related_music_genres.update(self.genre_mapping[genre])
        return list(related_music_genres)

    def recommend_music_based_on_movie(self, movie_title: str, num_recommendations: int = 5) -> Union[List[Dict[str, str]], str]:
        movie = self.find_movie(movie_title)
        if movie is None:
            return f"Movie not found. Did you mean one of these: {', '.join(get_close_matches(movie_title, self.movies_df['title'].tolist(), n=3))}?"

        movie_genres = movie['genres']
        related_music_genres = self.get_related_music_genres(movie_genres)

        if not related_music_genres:
            return "No matching music genres found for this movie's genres."

        matching_music = self.music_genres_df[
            self.music_genres_df['genres'].str.lower().isin([g.lower() for g in related_music_genres])
        ]

        if matching_music.empty:
            return "No music recommendations found for the related genres."

        recommendations = matching_music.sample(n=min(num_recommendations, len(matching_music)))
        return recommendations[['genres', 'track_names']].to_dict('records')

def main():
    st.title("Movie-to-Music Recommendation System")
    st.sidebar.header("Settings")
    movies_path = st.sidebar.text_input("Movies Dataset Path", "tmdb_5000_movies.csv")
    music_path = st.sidebar.text_input("Music Dataset Path", "extended_data_by_genres.csv")
    recommender = GenreRecommendationSystem(movies_path=movies_path, music_path=music_path)

    st.write("Enter a movie title to get music recommendations based on its genre!")

    movie_title = st.text_input("Movie Title", "")
    if movie_title:
        recommendations = recommender.recommend_music_based_on_movie(movie_title)

        if isinstance(recommendations, str):
            st.error(recommendations)
        else:
            st.subheader(f"Recommended music tracks for '{movie_title}':")
            for i, rec in enumerate(recommendations, 1):
                st.write(f"{i}. Genre: {rec['genres']} - Track: {rec['track_names']}")
    else:
        st.write("Please enter a movie title to get recommendations.")

if __name__ == "__main__":
    main()


Writing movie_music_recommender.py


In [32]:
from pyngrok import ngrok
import os

# Get your authtoken from https://dashboard.ngrok.com/auth
# and replace 'YOUR_AUTHTOKEN' with your actual token
ngrok.set_auth_token("2qbpqjKwLPYYHXL3hgHMYnwGC69_EeJdHo22u5SjzUxxXDq7")

# Expose the Streamlit app on a public URL
# Use 'addr' instead of 'port' to specify the address and port to forward to.
public_url = ngrok.connect(addr='8501')

# Run the Streamlit app in the background
os.system('streamlit run movie_music_recommender.py &')

# Print the public URL
public_url

<NgrokTunnel: "https://19eb-35-185-26-254.ngrok-free.app" -> "http://localhost:8501">